In [15]:
### Test linear SVM
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC

In [32]:
data1 = pd.read_csv('data1_t_scaled_250_03.csv', index_col = 0, header = 0)
data2 = pd.read_csv('data2_t_scaled_250_03.csv', index_col = 0, header = 0)

labels1 = pd.read_csv('5k_v3_labels.csv', index_col = 0, header = 0)
labels2 = pd.read_csv('5k_v3_nextgem_labels.csv', index_col = 0, header = 0)

xx = np.where(labels1 != 'CD3+ CD14+ cell')[0]
labels1 = labels1.iloc[xx]

xx = np.where(labels2 != 'CD3+ CD14+ cell')[0]
labels2 = labels2.iloc[xx]

xx2 = np.where(labels1 == 'NC/Int Monocyte')[0]
labels1.iloc[xx2] = 'NC-Int Monocyte'

xx2 = np.where(labels2 == 'NC/Int Monocyte')[0]
labels2.iloc[xx2] = 'NC-Int Monocyte'

In [17]:
len(labels1)

3817

In [18]:
clf = LinearSVC()
clf.fit(data1, labels1)

C:\Users\lcmmichielsen\Anaconda3\envs\Seurat\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\lcmmichielsen\Anaconda3\envs\Seurat\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [20]:
pred = clf.predict(data2)

In [21]:
np.unique(pred)

array(['B cell', 'C Monocyte', 'CD4+ T cell', 'CD8+ T cell',
       'NC/Int Monocyte', 'NK cell', 'NKT cell', 'cDC', 'pDC'],
      dtype=object)

In [26]:
np.sum(pred == np.squeeze(labels2.values))

3602

In [24]:
np.shape(pred)

(4011,)

In [25]:
np.shape(labels2.values)

(4011, 1)

### Now we use one-class/linear SVM with hierarchy

In [27]:
from newick import *
from build_classifier import *
from predict_labels import *

In [28]:
### Create the tree
tree = loads('(((cDC, pDC)DC, (C Monocyte, NC-Int Monocyte)Monocyte)Myeloid, (B cell, (NK cell, NKT cell, (CD8+ T cell, CD4+ T cell, gd T cell)T cell)NK and T cell)Lymphoid)root')

In [53]:
### Train the tree
# tree = train_tree(data1, labels1, tree, classifier = 'svm_occ', dimred = True)
tree = train_tree(data1, labels1, tree, classifier = 'svm', dimred = False)

In [54]:
### Predict the labels
pred = predict_labels(data2, tree)

In [55]:
np.sum(pred == np.squeeze(labels2.values))

3265

In [49]:
np.unique(pred)

array(['B cell', 'C Monocyte', 'CD4+ T cell', 'CD8+ T cell', 'DC',
       'Lymphoid', 'Monocyte', 'Myeloid', 'NC-Int Monocyte',
       'NK and T cell', 'NK cell', 'NKT cell', 'T cell', 'cDC',
       'gd T cell', 'pDC', 'root'], dtype='<U15')

In [44]:
np.unique(labels2.values[np.where(pred == 'T cell')[0]])

array(['CD4+ T cell', 'CD8+ T cell', 'gd T cell'], dtype=object)

In [50]:
len(np.where(pred == 'root')[0])

168

In [51]:
def confusion(y_true, y_pred):
    '''
    Construct a confusion matrix
    '''
    
    # Construct normalized confusion matrix
    num_cluster = len(np.unique(y_true))
    num_pred = len(np.unique(y_pred))
    NC = np.zeros([num_cluster,num_pred], dtype = int)

    for i, group_true in enumerate(np.unique(y_true)):
        a = y_true == group_true
        a = np.squeeze(a)
        for j, group_pred in enumerate(np.unique(y_pred)):
            b = y_pred == group_pred
            b = np.squeeze(b)
            NC[i,j] = sum(a & b)

    NC = pd.DataFrame(NC, columns = np.unique(y_pred), index = np.unique(y_true))
    
    return NC


In [56]:
confusion(labels2.values, pred)

,B cell,C Monocyte,CD4+ T cell,CD8+ T cell,Lymphoid,Monocyte,Myeloid,NC-Int Monocyte,NK and T cell,NK cell,NKT cell,T cell,cDC,pDC,root
B cell,273,4,0,0,1,0,4,0,0,0,0,0,0,0,57
C Monocyte,0,1112,0,0,0,9,4,7,0,0,0,0,2,0,37
CD4+ T cell,0,0,1426,12,15,0,0,0,10,0,3,2,0,0,71
CD8+ T cell,0,0,244,90,2,0,0,0,1,11,1,1,0,0,13
NC-Int Monocyte,0,12,0,0,0,0,1,42,0,0,0,0,0,0,4
NK cell,0,0,0,18,22,0,1,0,0,288,3,0,0,0,5
NKT cell,0,0,0,0,0,0,0,0,0,5,7,0,0,0,8
cDC,0,18,0,0,0,1,0,0,0,0,0,0,18,0,55
gd T cell,0,0,60,7,0,0,0,0,0,0,0,0,0,0,2
pDC,0,0,0,0,0,0,0,0,0,0,0,0,1,9,12
